<a href="https://colab.research.google.com/github/Anusha-DeviE/College-Chatbot/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faiss-cpu transformers accelerate sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer



csv_path = '/content/Chatbot_data.csv'
try:
    df = pd.read_csv(csv_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, encoding='latin-1')

df = df.dropna()
df.columns = [col.strip().lower() for col in df.columns]
print("✅ Dataset Loaded. Sample:")
print(df.head())

✅ Dataset Loaded. Sample:
                                     queries  \
0  What courses are offered at your college?   
1  Can you tell me about the B.Tech program?   
2    What is the duration of the MBA course?   
3               How long is the MCA program?   
4   Is M.Tech available at your institution?   

                                             answers  
0    We offer B.Tech, M.Tech, MBA, and MCA programs.  
1  B.Tech is a 4-year undergraduate engineering p...  
2      The MBA course is a 2-year full-time program.  
3                   The MCA program is 3 years long.  
4  Yes, we offer a 2-year M.Tech program in multi...  


In [ ]:
#vector Embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')
questions = df['queries'].tolist()
corpus_embeddings = embedder.encode(questions, convert_to_numpy=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
#Indexing
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

In [ ]:
#LLM
generator = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    tokenizer="tiiuae/falcon-7b-instruct",
    device=0
    if torch.cuda.is_available()
    else -1,
    max_new_tokens=200
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
generator.model.save_pretrained("./falcon-7b-instruct/")
generator.tokenizer.save_pretrained("./falcon-7b-instruct/")


('./falcon-7b-instruct/tokenizer_config.json',
 './falcon-7b-instruct/special_tokens_map.json',
 './falcon-7b-instruct/chat_template.jinja',
 './falcon-7b-instruct/tokenizer.json')

In [ ]:
def get_response(user_query, top_k=1):
    embedding = embedder.encode([user_query], convert_to_numpy=True)
    distances, indices = index.search(embedding, top_k)
    retrieved_answer = df.iloc[indices[0][0]]['answers']

    prompt = f"""You are a helpful assistant that answers college admission queries.
Relevant info: {retrieved_answer}
Question: {user_query}
Answer:"""

    generated = generator(prompt)[0]['generated_text']
    response = generated.split("Answer:")[-1].strip()
    return response

In [ ]:
print("\n Chatbot is ready! (type 'exit' to quit):\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print(" Chatbot: Goodbye!")
        break
    reply = get_response(user_input)
    print(" Chatbot:", reply)


 Chatbot is ready! (type 'exit' to quit):

You: what is the admission process


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Chatbot:,,, The admission process typically begins in April every year. It consists of submitting your application, SAT or ACT scores, and any other required documents. Some schools may also require essays and recommendations. It's important to research each school's specific application requirements to ensure a smooth process.
You: what are the courses present


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Chatbot:,,, We have a range of courses in various domains such as engineering, management, computer applications, and more.
You: who is the dean of FEt


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Chatbot:,,, The current Dean of FEt at our college is Dr. S.K. Reddy.
You: is imag processing course taught for the computer science


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Chatbot:,,, Yes, we do offer a course in computer vision and image processing for our computer science students.


In [ ]:
import gradio as gr

def chatbot_interface(user_query):
    return get_response(user_query)

gr.Interface(
    fn=chatbot_interface,
    inputs="text",
    outputs="text",
    title="College Inquiry Chatbot",
    description="Ask anything about college admissions, fees, or programs!"
).launch()
